# Obtain and Append Data

This notebook contains code to obtain and append financial data to avoid going over API request limits.

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import plotly.graph_objects as go

import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import numpy as np
from IPython.display import clear_output
import time
# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [2]:
import platform
print(platform.python_version())

3.8.3


In [3]:
# Obtain the API credentials from an external hidden file

from passwords import api_key

In [4]:
# A function to obtain price history

#Utilizing "UniBit API" by Vincent Jiang
#Site: https://unibit.ai/product

#Errors may occur if credits run out, add an error code check to this function

def retrieve_ticker_price_info(ticker, start_date, end_date = False):
    
    # Obtain today's date
    today = date.today()
    
    # A condition to set the end date to today if there is no end date given
    if not end_date:
        end_date = today
    
    
    url = 'https://api.unibit.ai/v2/stock/historical/?tickers={}&interval=1&startDate={}&endDate={}&selectedFields=all&dataType=json&accessKey={}'.format(ticker, start_date, end_date, api_key)
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print("The response status code is: " + str(response.status_code))
        return None
    
    return response.json()
    

In [5]:
def retrieve_price_history_dataframe(ticker, ticker_price_history):
    
    data_frame = pd.DataFrame(ticker_price_history['result_data'][ticker])
    
    # Set the index to the 'date' column
    data_frame.set_index('date', inplace = True)
    
    data_frame.sort_index(inplace=True)
    
    return data_frame

In [6]:
today = date.today()
a_year_ago = str(date.today() - timedelta(days=1*365))
some_five_years_ago = str(date.today() - timedelta(days=5*365))

## Obtain Original Data from the API

**NOTE: Skip this section if you already have a data set**

In [7]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
inx_info = retrieve_ticker_price_info('^GSPC', some_five_years_ago)

# Create and view the dataframe
inx_df = retrieve_price_history_dataframe('^GSPC', inx_info)
inx_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2015-08-31,3915100000,1986.73,1965.98,1972.18,1972.18,1986.73
2015-09-01,4371850000,1970.09,1903.07,1913.85,1913.85,1970.09
2015-09-02,3742620000,1948.91,1916.52,1948.86,1948.86,1916.52
2015-09-03,3520700000,1975.01,1944.72,1951.13,1951.13,1950.79
2015-09-04,3167090000,1947.76,1911.21,1921.22,1921.22,1947.76


In [8]:
# Save the data as a csv
# Keep the index as it contains the dates
#inx_df.to_csv('financial_data/GSPC.csv')
# five years
inx_df.to_csv('financial_data/august_31_2015_to_now_GSPC.csv')

## Import a CSV of Time Series Data

This code assumes the data is in ascending order, if this is not the case then make the relevant adjustments. 

In [7]:
inx_df = pd.read_csv('financial_data/august_31_2015_to_now_GSPC.csv', index_col = 'date')

# Convert index to date
# Comment this out if the index is already date type
inx_df.index = pd.to_datetime(inx_df.index).date

inx_df.head()

,volume,high,low,adj_close,close,open
2015-08-31,3915100000,1986.73,1965.98,1972.18,1972.18,1986.73
2015-09-01,4371850000,1970.09,1903.07,1913.85,1913.85,1970.09
2015-09-02,3742620000,1948.91,1916.52,1948.86,1948.86,1916.52
2015-09-03,3520700000,1975.01,1944.72,1951.13,1951.13,1950.79
2015-09-04,3167090000,1947.76,1911.21,1921.22,1921.22,1947.76


In [8]:
# Obtain the most recent date
inx_df.index.max()

datetime.date(2020, 8, 27)

## Obtain Most Recent Data From The API

In [9]:
# Obtain the next day after the series ends, as a string
next_day_series_ends = str(inx_df.index.max() + timedelta(days=1))
next_day_series_ends

'2020-08-28'

In [10]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
additional_info = retrieve_ticker_price_info('^GSPC', next_day_series_ends)

# Create and view the dataframe
additional_info_df = retrieve_price_history_dataframe('^GSPC', additional_info)
additional_info_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2020-08-28,3855880000,3509.23,3484.32,3508.01,3508.01,3494.69
2020-08-31,2543021541,3514.77,3493.25,3500.31,3500.31,3509.73
2020-09-01,2245334277,3528.03,3494.60,3526.65,3526.65,3507.44
2020-09-02,2581524590,3588.11,3535.23,3580.84,3580.84,3543.76
2020-09-03,3193601820,3564.85,3427.41,3455.06,3455.06,3564.74


In [11]:
additional_info_df.tail()

,volume,high,low,adj_close,close,open
date,,,,,,
2020-10-05,2040147536,3409.57,3367.27,3408.63,3408.63,3367.27
2020-10-06,2549972990,3431.56,3354.54,3360.95,3360.95,3408.74
2020-10-07,2074262423,3426.26,3384.56,3419.45,3419.45,3384.56
2020-10-08,2088859740,3447.28,3428.15,3446.83,3446.83,3434.28
2020-10-09,2227950074,3482.34,3458.07,3477.13,3477.13,3459.67


## Append The Original Dataframe

You can either add to the old file or create a new one

In [12]:
result = pd.concat([inx_df, additional_info_df])

In [13]:
result.tail(15)

,volume,high,low,adj_close,close,open
2020-09-21,2973366814,3285.57,3229.10,3281.06,3281.06,3285.57
2020-09-22,2364817767,3320.31,3270.95,3315.57,3315.57,3295.75
2020-09-23,2462124190,3323.35,3232.57,3236.92,3236.92,3320.11
2020-09-24,2386884281,3278.70,3209.45,3246.59,3246.59,3226.14
2020-09-25,2180313684,3306.88,3228.44,3298.46,3298.46,3236.66
2020-09-28,2221021352,3360.74,3332.91,3351.60,3351.60,3333.90
2020-09-29,1945022669,3357.92,3327.54,3335.47,3335.47,3350.92
2020-09-30,2616747156,3393.56,3340.47,3363.00,3363.00,3341.21
2020-10-01,2355862321,3397.18,3361.39,3380.80,3380.80,3385.87
2020-10-02,2350090813,3369.10,3323.69,3348.44,3348.44,3338.94


## Save The New Dataframe

You can overwrite the old file as well

In [14]:
# Save the data as a csv
result.to_csv('financial_data/GSPC_newdata.csv', index_label = 'date')

In [16]:
# Read in the file
pd.read_csv('financial_data/GSPC_newdata.csv', index_col = 'date').tail(15)

,volume,high,low,adj_close,close,open
date,,,,,,
2020-09-21,2973366814,3285.57,3229.10,3281.06,3281.06,3285.57
2020-09-22,2364817767,3320.31,3270.95,3315.57,3315.57,3295.75
2020-09-23,2462124190,3323.35,3232.57,3236.92,3236.92,3320.11
2020-09-24,2386884281,3278.70,3209.45,3246.59,3246.59,3226.14
2020-09-25,2180313684,3306.88,3228.44,3298.46,3298.46,3236.66
2020-09-28,2221021352,3360.74,3332.91,3351.60,3351.60,3333.90
2020-09-29,1945022669,3357.92,3327.54,3335.47,3335.47,3350.92
2020-09-30,2616747156,3393.56,3340.47,3363.00,3363.00,3341.21
2020-10-01,2355862321,3397.18,3361.39,3380.80,3380.80,3385.87
